In [ ]:
import numpy as np
import pandas as pd
import requests
import time
import os

# Ejemplo de uso de la función para obtener nuevos recording_ids

`df = pd.read_csv(ruta_al_dataset_a_procesar.csv)`

`get_recording_id(df)`

# Función para obtener nuevos recording_ids

In [1]:
def get_recording_id(df):
    # Función para obtener el recording_id desde la API de MusicBrainz
    def get_recording_id(title, artist):
        query = f'recording:"{title}" AND artist:"{artist}"'
        url = f"https://musicbrainz.org/ws/2/recording/?query={query}&fmt=json"

        try:
            response = requests.get(url, timeout=10)  # Agregar timeout para evitar bloqueos indefinidos
            if response.status_code == 200:
                data = response.json()
                if "recordings" in data and len(data["recordings"]) > 0:
                    return data["recordings"][0]["id"]
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error on search '{title}' - '{artist}': {e}")
            return None

    # Rutas de los archivos de salida
    found_file = "../data/raw/nuevos_ids_encontrados.csv.temp"
    not_found_file = "../data/raw/ids_no_encontrados.csv.temp"

    # Crear los archivos si no existen
    # En caso de que ya existan y tengan registros, en lugar de sobrescribirse, los nuevos se añaden al final
    if not os.path.exists(found_file):
        pd.DataFrame(columns=["artist_name", "song_name", "recording_id"]).to_csv(found_file, index=False)
    if not os.path.exists(not_found_file):
        pd.DataFrame(columns=["artist_name", "song_name"]).to_csv(not_found_file, index=False)

    # Inicializar contadores
    total_requests = 0

    # Procesar el DataFrame
    for index in df.index:
        row = df.loc[index]

        # Saltar filas con un recording_id ya existente y válido
        if pd.notna(row["recording_id"]) and row["recording_id"] != "Not found":
            continue

        total_requests += 1
        print(f"\n[Progress] Request {total_requests}/{len(df)}...")

        # Obtener el recording_id
        recording_id = get_recording_id(row["song_name"], row["artist_name"])

        # Procesar y guardar los resultados por cada request
        # Es un poco más lento pero resulta en menos fallos, por lo que es preferible
        if not recording_id:  # Si no se encuentra el recording_id, se almacenan los datos procesados en un dataset ids_no_encontrados.csv.temp
            print(f"  ⚠️ Not Recording ID found for: {row['song_name']} - {row['artist_name']}")
            not_found_df = pd.DataFrame([{"artist_name": row["artist_name"], "song_name": row["song_name"]}])
            not_found_df.to_csv(not_found_file, mode="a", index=False, header=False)
        else:  # Si se encuentra el recording_id, se almacenan los datos procesados en un dataset nuevos_ids_encontrados.csv.temp
            print(f"  ✅ Recording ID found: {recording_id}")
            found_df = pd.DataFrame([{
                "artist_name": row["artist_name"],
                "song_name": row["song_name"],
                "recording_id": recording_id
            }])
            found_df.to_csv(found_file, mode="a", index=False, header=False)


        # Respetar el límite de velocidad de la API
        time.sleep(1)

    print(f"\nProcessing complete. Results saved in:\n - Found: {found_file}\n - Not Found: {not_found_file}")

# Obtener nuevos recording_id

In [ ]:
df = pd.read_csv('ruta_del_dataset_a_procesar.csv')

In [ ]:
get_recording_id(df)

Una vez termine o se corte la ejecución, los datos de los que se ha encontrado recording_id estarán en [nuevos_ids_encontrados.csv.temp](../data/raw/nuevos_ids_encontrados.csv.temp) y los que no se encuentren, en [ids_no_encontrados.csv.temp](../data/raw/ids_no_encontrados.csv.temp).

En caso de que se vuelva a ejecutar la función, si los archivos ya existen y contienen datos, estos no se sobreescriben, sino que se añaden al final. Para evitar duplicados y ahorrar tiempo, lo ideal es eliminar los datos que ya están en [nuevos_ids_encontrados.csv.temp](../data/raw/nuevos_ids_encontrados.csv.temp) y [ids_no_encontrados.csv.temp](../data/raw/ids_no_encontrados.csv.temp) del dataset a procesar. Para ello tenemos la función [remove_existing_records()](eliminar-de-un-dataset-que-existen-en-otro.ipynb)